# Module 1

In [3]:
from utilities.ingest import ETL
import pandas as pd

e = ETL()

e.main()

100%|██████████| 112/112 [00:38<00:00,  2.88it/s]


,stock_id,time_id,target,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,price,size,order_count
0,0,5,0.004136,21,1.001422,1.002818,1.001370,1.002922,3,30,2,100,1.002301,326,12
1,0,11,0.001445,0,0.999473,1.000176,0.999423,1.000376,205,100,20,30,0.999824,2,2
2,0,16,0.002168,58,0.999641,1.000503,0.999210,1.000551,101,90,186,300,1.000215,2,1
3,0,31,0.002195,3,0.999815,1.000463,0.999769,1.000509,343,100,5,142,1.000231,13,1
4,0,62,0.001747,16,0.999790,1.000210,0.999557,1.000303,101,17,2,100,1.000023,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
428908,126,32751,0.003461,22,0.999673,1.000457,0.999641,1.000620,100,11,1,6,0.999869,1,1
428909,126,32753,0.003113,0,1.000620,1.001275,1.000586,1.001310,120,3,100,103,1.000713,238,16
428910,126,32758,0.004070,57,0.999852,1.000839,0.999803,1.001086,300,1,101,100,1.000543,105,4
428911,126,32763,0.003357,8,1.000757,1.001350,1.000691,1.001613,100,100,100,7,1.001114,397,7


In [4]:
e = ETL(stock_id = "")
sample_sub = pd.read_csv('./data/sample_submission.csv')
book_train, trade_train, book_test, trade_test = e.ingest_all_parquet()

display(sample_sub.head())
display(book_train.head())
display(trade_train.head())

,row_id,target
0,0-4,0.003048
1,0-32,0.003048
2,0-34,0.003048


,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2,stock_id
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100,0
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100,0
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100,0
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100,0


,time_id,seconds_in_bucket,price,size,order_count,stock_id
0,5,21,1.002301,326,12,0
1,5,46,1.002778,128,4,0
2,5,50,1.002818,55,1,0
3,5,57,1.003155,121,5,0
4,5,68,1.003646,4,1,0


In [5]:
import numpy as np

# Applying feature engineering to get feature engineering artifact

class FeatureEngineering:
    
        # Function to calculate first WAP
    def calc_wap1(df):
        wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
        return wap

    # Function to calculate second WAP
    def calc_wap2(df):
        wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
        return wap

    def calc_wap3(df):
        wap = (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
        return wap

    def calc_wap4(df):
        wap = (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])
        return wap

    # Function to calculate the log of the return
    # Remember that logb(x / y) = logb(x) - logb(y)
    def log_return(series):
        return np.log(series).diff()

    # Calculate the realized volatility
    def realized_volatility(series):
        return np.sqrt(np.sum(series**2))

    # Function to count unique elements of a series
    def count_unique(series):
        return len(np.unique(series))
    
    
    def add_volume(self):
        pass

    def buy_sell_ratio(self):
        pass

    def add_ask_to_bid_size_ratio(self, df):
        """Ask size to bid size ratio"""
        df["ask_to_bid_size_ratio"] = (df["ask_size1"] + df["ask_size2"]) / (
            df["bid_size1"] + df["bid_size2"]
        )
        return df

    def add_second_to_first_ratio(self, df):
        """Ratios of second bids to first bids.
        
        Are there a lot of bids on the second row but not on the first row? Signals how much buying pressure is around the market price but not at it. 
        """
        df["buyside_second_ratio"] = (df["ask_size2"] * df["ask_price2"]) / (
            df["ask_size1"] * df["ask_price1"]
        )
        df["sellside_second_ratio"] = (df["bid_size2"] * df["bid_price2"]) / (
            df["bid_size1"] * df["bid_price1"]
        )
        df["second_row_tension_metric"] = (df["ask_size2"] * df["ask_price2"]) / (
            df["bid_size1"] * df["bid_price1"]
        )

        return df

    def group_to_feature(self, df):
        """Capture Features """
        feature_sum = [
            "seconds_in_bucket",
            "bid_price1",
            "ask_price1",
            "bid_price2",
            "ask_price2",
            "bid_size1",
            "ask_size1",
            "bid_size2",
            "ask_size2",
        ]
        index = ["mean", "std", "min", "25%", "50%", "75%", "max"]
        val1 = df[feature_sum].describe().loc[index].values.reshape(-1)
        #     val2 = np.sum(df[feature_sum]).values
        return val1
    

    def merge(self, curr_book, curr_trade):
        """ This return a merged dataframe of trades where the trades actually took place 
        
        BUG: I am having trouble making this work. Restart from first principles.
        """
        merged_data = pd.merge(
            curr_book, curr_trade, on=["time_id", "seconds_in_bucket"]
        )

        return merged_data


class BookPreprocessor:
    
    def __init__(self, fe: FeatureEngineering):
        
        self.fe = fe
        
        # Dict for aggregations
        self.create_feature_dict = {
            'wap1': [np.sum, np.std],
            'wap2': [np.sum, np.std],
            'wap3': [np.sum, np.std],
            'wap4': [np.sum, np.std],
            'log_return1': [self.fe.realized_volatility],
            'log_return2': [self.fe.realized_volatility],
            'log_return3': [self.fe.realized_volatility],
            'log_return4': [self.fe.realized_volatility],
            'wap_balance': [np.sum, np.max],
            'price_spread':[np.sum, np.max],
            'price_spread2':[np.sum, np.max],
            'bid_spread':[np.sum, np.max],
            'ask_spread':[np.sum, np.max],
            'total_volume':[np.sum, np.max],
            'volume_imbalance':[np.sum, np.max],
            "bid_ask_spread":[np.sum,  np.max],
        }
        self.create_feature_dict_time = {
            'log_return1': [self.fe.realized_volatility],
            'log_return2': [self.fe.realized_volatility],
            'log_return3': [self.fe.realized_volatility],
            'log_return4': [self.fe.realized_volatility],
        }
    
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(self, fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    
    def main(self, file_path):
        '''End to end pipeline'''
        df = pd.read_parquet(file_path)
        # Calculate Wap
        df['wap1'] = self.fe.calc_wap1(df)
        df['wap2'] = self.fe.calc_wap2(df)
        df['wap3'] = self.fe.calc_wap3(df)
        df['wap4'] = self.fe.calc_wap4(df)
        # Calculate log returns
        df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(self.fe.log_return)
        df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(self.fe.log_return)
        df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(self.fe.log_return)
        df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(self.fe.log_return)
        # Calculate wap balance
        df['wap_balance'] = abs(df['wap1'] - df['wap2'])
        # Calculate spread
        df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
        df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
        df['bid_spread'] = df['bid_price1'] - df['bid_price2']
        df['ask_spread'] = df['ask_price1'] - df['ask_price2']
        df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
        df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
        df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
        
    
        # Create row_id so we can merge
        stock_id = file_path.split('=')[1]
        df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
        df_feature.drop(['time_id_'], axis = 1, inplace = True)
        return df_feature
        
    def apply(self, create_feature_dict, create_feature_dict_time):

        # Get the stats for different windows
        df_feature = self.get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
        df_feature_500 = self.get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
        df_feature_400 = self.get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
        df_feature_300 = self.get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
        df_feature_200 = self.get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
        df_feature_100 = self.get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)

        df_feature = self.merge(df_feature, df_feature_500, df_feature_400, df_feature_300, df_feature_200, df_feature_100)
        
        return df_feature
        
        
    def merge(self, df_feature, df_feature_500, df_feature_400, df_feature_300, df_feature_200, df_feature_100):
        # Merge all
        df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
        df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
        df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
        df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
        df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
        # Drop unnecesary time_ids
        df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
        
        return df_feature



In [ ]:
bp = BookPreprocessor(fe = FeatureEngineering)

bp.main('/Users/mtaruno/Documents/DevZone/optiver/data/book_train.parquet')